#  최종파일 작성

```
１. csv 파일 읽기
２. 'Country_Region','Confirmed' 두 개의 컬럼만 가져오기
３. 'Confirmed' 에 데이터가 없는 행 삭제하기
４. 'Country_Region'의 국가명을 여러 파일에 일관되게 변경하기
５. 'Confirmed' 데이터 타입을 int64(정수)로 변경
６. 'Country_Region' 를 기준으로 중복된 데이터를 합치기
７. 파일명을 기반으로 날짜 문자열 변환하고, 'Confirmed' 컬럼명 변경하기
```

## 라이브러리 로드 및 함수 정의

In [4]:
import pandas as pd
import json

In [5]:
PATH = "../data/covid/covid_19_daily_reports/"

In [6]:
# 국가명을 가지고 있는 json 로드

with open("../data/covid/country_convert.json", "r",
          encoding="utf-8-sig") as json_file:
    json_data = json.load(json_file)

In [8]:
# 국가명 변경하기 함수 작성

def counrty_name_convert(row):
    if (row["Country_Region"] in json_data):
        return json_data[row["Country_Region"]]  # 바뀐 행 리턴
    return row["Country_Region"]  # 원본 행 리턴

In [9]:
# 컬럼명 추출 후 컬럼명 변경, nan 데이터 정리


def create_dateframe(filename):
    doc = pd.read_csv(PATH + filename, encoding="utf-8-sig")
    try:
        doc = doc[["Country_Region", "Confirmed"]]
    except:
        doc = doc[["Country/Region", "Confirmed"]]
        # 컬럼명을 원하는 컬럼명으로 변경
        doc.columns = ["Country_Region", "Confirmed"]

    # 확진자 수 없는 것 제거
    doc = doc.dropna(subset=["Confirmed"])
    
    # 확진자 수 컬럼 타입 변경
    doc = doc.astype({"Confirmed": "int64"})
    
    # Country_Region 국가명 변경하기 호출
    doc["Country_Region"] = doc.apply(counrty_name_convert, axis=1)
    
    # 국가명으로 그룹 잡아 합계 구하기
    doc = doc.groupby("Country_Region").sum()

    # 파일명에서 날짜 부분 추출
    date_column = (filename.split(".")[0].lstrip("0").replace("-","/",))
    # 확진자 수 컬럼을 날짜로 변경
    doc.columns = [date_column]
    return doc

In [10]:
# 테스트 해보기

doc1 = create_dateframe("01-22-2020.csv")
doc2 = create_dateframe("04-07-2020.csv")

doc = pd.merge(doc1, doc2, how="outer", left_index=True, right_index=True)
# N/A => 0
doc = doc.fillna(0)
doc.head()

,1/22/2020,4/07/2020
Country_Region,,
Afghanistan,0.0,423
Albania,0.0,383
Algeria,0.0,1468
Andorra,0.0,545
Angola,0.0,17


## 하나의 파일로 합치기

* data/covid/covid_19_daily_reports 안의 csv를 읽어와서 하나의 파일로 합치는 작업

In [13]:
# 사전개념

import os

(file_list, csv_list) = (os.listdir(PATH), list())

for file in file_list:
        if file.split(".")[-1] == "csv":
            csv_list.append(file)

# 날짜별로 데이터가 들어가야 하기 때문에
csv_list.sort()
csv_list

['01-22-2020.csv',
 '01-23-2020.csv',
 '01-24-2020.csv',
 '01-25-2020.csv',
 '01-26-2020.csv',
 '01-27-2020.csv',
 '01-28-2020.csv',
 '01-29-2020.csv',
 '01-30-2020.csv',
 '01-31-2020.csv',
 '02-01-2020.csv',
 '02-02-2020.csv',
 '02-03-2020.csv',
 '02-04-2020.csv',
 '02-05-2020.csv',
 '02-06-2020.csv',
 '02-07-2020.csv',
 '02-08-2020.csv',
 '02-09-2020.csv',
 '02-10-2020.csv',
 '02-11-2020.csv',
 '02-12-2020.csv',
 '02-13-2020.csv',
 '02-14-2020.csv',
 '02-15-2020.csv',
 '02-16-2020.csv',
 '02-17-2020.csv',
 '02-18-2020.csv',
 '02-19-2020.csv',
 '02-20-2020.csv',
 '02-21-2020.csv',
 '02-22-2020.csv',
 '02-23-2020.csv',
 '02-24-2020.csv',
 '02-25-2020.csv',
 '02-26-2020.csv',
 '02-27-2020.csv',
 '02-28-2020.csv',
 '02-29-2020.csv',
 '03-01-2020.csv',
 '03-02-2020.csv',
 '03-03-2020.csv',
 '03-04-2020.csv',
 '03-05-2020.csv',
 '03-06-2020.csv',
 '03-07-2020.csv',
 '03-08-2020.csv',
 '03-09-2020.csv',
 '03-10-2020.csv',
 '03-11-2020.csv',
 '03-12-2020.csv',
 '03-13-2020.csv',
 '03-14-2020

In [14]:
# 위의 개념을 함수로 작성하기

def generate_dateframe_by_path(PATH):
    # 변수 선언
    (file_list, csv_list) = (os.listdir(PATH), list())

    first_doc = True

    for file in file_list:
        if file.split(".")[-1] == "csv":
            csv_list.append(file)

    # 날짜별로 데이터가 들어가야 하기 때문에
    csv_list.sort()

    # file 명이 들어있는 리스트에서 하나씩 파일명을 가져와서 create_dateframe()으로 전처리 작업하고
    # 첫번째 문서의 경우에는 이전 문서와 합할 문서가 없기 때문에 if 문 실행
    # 두번째 문서부터는 else 실행하여 merge 실행
    for file in csv_list:
        # 국가명과 확진자 수만 추출하여 문서 작성
        doc = create_dateframe(file)
        if first_doc:
            (final_doc, first_doc) = (doc, False)
        else:
            final_doc = pd.merge(final_doc,
                                 doc,
                                 how="outer",
                                 left_index=True,
                                 right_index=True)

    final_doc = final_doc.fillna(0)
    return final_doc

In [15]:
# 이제까지 작업 테스트

doc = generate_dateframe_by_path(PATH)

# 0.0 으로 나오는 데이터 0 으로 변경
doc = doc.astype("int64")
doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,7/19/2020,7/20/2020,7/21/2020,7/22/2020,7/23/2020,7/24/2020,7/25/2020,7/26/2020,7/27/2020,7/28/2020
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,35475,35526,35615,35727,35928,35981,36036,36157,36263,36368
Albania,0,0,0,0,0,0,0,0,0,0,...,4090,4171,4290,4358,4466,4570,4637,4763,4880,4997
Algeria,0,0,0,0,0,0,0,0,0,0,...,23084,23691,24278,24872,25484,26159,26764,27357,27973,28615
Andorra,0,0,0,0,0,0,0,0,0,0,...,880,884,884,889,889,897,897,897,907,907
Angola,0,0,0,0,0,0,0,0,0,0,...,705,749,779,812,851,880,916,932,950,1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,0,2,2,2,2,2,2,2,2,2,...,383,384,401,408,412,415,417,420,431,446
West Bank and Gaza,0,0,0,0,0,0,0,0,0,0,...,8549,8916,9228,9398,9744,10093,10306,10469,10621,10938
Yemen,0,0,0,0,0,0,0,0,0,0,...,1606,1619,1629,1640,1654,1674,1674,1681,1691,1703


In [16]:
# 최종 완성된 doc 데이터프레임을 csv 파일로 저장
doc.to_csv("../data/covid/final_df.csv")

# 그래프 작성

https://app.flourish.studio/@flourish/bar-chart-race

* 구글 계정으로 로그인
* 데이터를 원하는 형식으로 넣어주면 차트를 그려줌

* 최종 파일을 읽어와서 iso2 값 작업 후 링크 주소 넣어주기

## 최종 파일을 읽기

In [76]:
df_confirmed = pd.read_csv("../data/covid/final_df.csv")
df_confirmed.shape

(187, 190)

## 국가명과 iso2 값 추출

In [77]:
country_info = pd.read_csv("../data/covid/UID_ISO_FIPS_LookUp_Table.csv",encoding="utf-8-sig")
country_info.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.20270,17.873900,Angola,32866268.0


In [78]:
# 국가명이 Namibia 의 정보 추출 : iso2가 NaN

country_info[country_info["Country_Region"] == "Namibia"]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
365,516,NaN,NAM,516.0,NaN,NaN,NaN,Namibia,-22.9576,18.4904,Namibia,2540916.0


In [79]:
# null 정보 확인
country_info.isnull().sum()

UID                 0
iso2                3
iso3                2
code3               2
FIPS              771
Admin2            812
Province_State    188
Country_Region      0
Lat               141
Long_             141
Combined_Key        0
Population        141
dtype: int64

In [80]:
# null 상태의 컬럼 확인

country_info[country_info["iso2"].isnull()]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
126,9999,NaN,NaN,NaN,NaN,NaN,NaN,Diamond Princess,NaN,NaN,Diamond Princess,NaN
364,8888,NaN,NaN,NaN,NaN,NaN,NaN,MS Zaandam,NaN,NaN,MS Zaandam,NaN
365,516,NaN,NAM,516.0,NaN,NaN,NaN,Namibia,-22.9576,18.4904,Namibia,2540916.0


* iso2 가 null 인 것을 확인해 본 결과 Diamond Princess, MS Zaanda 이렇게 배의 이름인 것들
* 무시해도 되는 상태

In [81]:
# iso2 값을 가지고 있는 파일 로드

country_info = pd.read_csv(
    "../data/covid/UID_ISO_FIPS_LookUp_Table.csv",
    encoding="utf-8-sig",
    keep_default_na=False,
    na_values="",
)
country_info.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
3,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0
4,24,AO,AGO,24.0,NaN,NaN,NaN,Angola,-11.20270,17.873900,Angola,32866268.0


In [82]:
# iso2 / Country_Region 컬럼만 추출

country_info = country_info[["iso2","Country_Region"]].copy()
country_info.head()

,iso2,Country_Region
0,AF,Afghanistan
1,AL,Albania
2,DZ,Algeria
3,AD,Andorra
4,AO,Angola


In [83]:
# 주 별로 되어 있기 때문에 나라별 추출을 위해 중복행 제거

country_info = country_info.drop_duplicates(subset="Country_Region",
                                            keep="last")
country_info.shape

(188, 2)

In [84]:
# 날짜별 국가별 확진자 수와 국가별 iso2 값 병합

doc_final_country = pd.merge(
    df_confirmed,
    country_info,
    how="left",
    on="Country_Region",
)
doc_final_country

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/20/2020,7/21/2020,7/22/2020,7/23/2020,7/24/2020,7/25/2020,7/26/2020,7/27/2020,7/28/2020,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,35526,35615,35727,35928,35981,36036,36157,36263,36368,AF
1,Albania,0,0,0,0,0,0,0,0,0,...,4171,4290,4358,4466,4570,4637,4763,4880,4997,AL
2,Algeria,0,0,0,0,0,0,0,0,0,...,23691,24278,24872,25484,26159,26764,27357,27973,28615,DZ
3,Andorra,0,0,0,0,0,0,0,0,0,...,884,884,889,889,897,897,897,907,907,AD
4,Angola,0,0,0,0,0,0,0,0,0,...,749,779,812,851,880,916,932,950,1000,AO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,Vietnam,0,2,2,2,2,2,2,2,2,...,384,401,408,412,415,417,420,431,446,VN
183,West Bank and Gaza,0,0,0,0,0,0,0,0,0,...,8916,9228,9398,9744,10093,10306,10469,10621,10938,PS
184,Yemen,0,0,0,0,0,0,0,0,0,...,1619,1629,1640,1654,1674,1674,1681,1691,1703,YE
185,Zambia,0,0,0,0,0,0,0,0,0,...,3326,3386,3583,3789,3856,4328,4481,4552,5002,ZM


In [85]:
# null 상태 갯수

doc_final_country.isnull().sum()

Country_Region    0
1/22/2020         0
1/23/2020         0
1/24/2020         0
1/25/2020         0
                 ..
7/25/2020         0
7/26/2020         0
7/27/2020         0
7/28/2020         0
iso2              4
Length: 191, dtype: int64

In [86]:
# null 상태의 컬럼 확인

doc_final_country[doc_final_country["iso2"].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/20/2020,7/21/2020,7/22/2020,7/23/2020,7/24/2020,7/25/2020,7/26/2020,7/27/2020,7/28/2020,iso2
44,Cruise Ship,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
49,Diamond Princess,0,0,0,0,0,0,0,0,0,...,712,712,712,712,712,712,712,712,712,NaN
105,MS Zaandam,0,0,0,0,0,0,0,0,0,...,9,9,9,9,9,9,9,9,9,NaN
130,Others,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [87]:
# 특정 컬럼에 없는 데이터 삭제하기
# 여기서 copy() 을 안하면 아래서 warning 발생

doc_final_country = doc_final_country.dropna(subset=["iso2"]).copy()
doc_final_country[doc_final_country["iso2"].isnull()]

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/20/2020,7/21/2020,7/22/2020,7/23/2020,7/24/2020,7/25/2020,7/26/2020,7/27/2020,7/28/2020,iso2


In [88]:
# 국기 링크를 기존 컬럼 기반해서 만들어 데이터프레임에 붙이기

# 국기 이미지를 얻어가기 위해 아래와 같은 주소가 필요함
# <img src="https://www.countryflags.io/be/flat/64.png">

def create_flag_link(row):
    flag_link = "https://www.countryflags.io/" + row + "/flat/64.png"
    return flag_link

In [89]:
# 적용하기

doc_final_country["iso2"] = doc_final_country["iso2"].map(create_flag_link)
doc_final_country.head()

,Country_Region,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,...,7/20/2020,7/21/2020,7/22/2020,7/23/2020,7/24/2020,7/25/2020,7/26/2020,7/27/2020,7/28/2020,iso2
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,35526,35615,35727,35928,35981,36036,36157,36263,36368,https://www.countryflags.io/AF/flat/64.png
1,Albania,0,0,0,0,0,0,0,0,0,...,4171,4290,4358,4466,4570,4637,4763,4880,4997,https://www.countryflags.io/AL/flat/64.png
2,Algeria,0,0,0,0,0,0,0,0,0,...,23691,24278,24872,25484,26159,26764,27357,27973,28615,https://www.countryflags.io/DZ/flat/64.png
3,Andorra,0,0,0,0,0,0,0,0,0,...,884,884,889,889,897,897,897,907,907,https://www.countryflags.io/AD/flat/64.png
4,Angola,0,0,0,0,0,0,0,0,0,...,749,779,812,851,880,916,932,950,1000,https://www.countryflags.io/AO/flat/64.png


In [90]:
# 실 데이터 컬럼의 위치를 앞으로 움직이기

# 현재 데이터 프레임을 list 로 변경
cols = doc_final_country.columns.tolist()

# 컬럼 위치 변경
# 마지막에 있는 iso2는 지우고
cols.remove("iso2")

# 두번째 열에 iso2 삽입
cols.insert(1,"iso2")

doc_final_country = doc_final_country[cols]
doc_final_country.head()

,Country_Region,iso2,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,...,7/19/2020,7/20/2020,7/21/2020,7/22/2020,7/23/2020,7/24/2020,7/25/2020,7/26/2020,7/27/2020,7/28/2020
0,Afghanistan,https://www.countryflags.io/AF/flat/64.png,0,0,0,0,0,0,0,0,...,35475,35526,35615,35727,35928,35981,36036,36157,36263,36368
1,Albania,https://www.countryflags.io/AL/flat/64.png,0,0,0,0,0,0,0,0,...,4090,4171,4290,4358,4466,4570,4637,4763,4880,4997
2,Algeria,https://www.countryflags.io/DZ/flat/64.png,0,0,0,0,0,0,0,0,...,23084,23691,24278,24872,25484,26159,26764,27357,27973,28615
3,Andorra,https://www.countryflags.io/AD/flat/64.png,0,0,0,0,0,0,0,0,...,880,884,884,889,889,897,897,897,907,907
4,Angola,https://www.countryflags.io/AO/flat/64.png,0,0,0,0,0,0,0,0,...,705,749,779,812,851,880,916,932,950,1000


In [92]:
# 컬럼명 변경

cols[1]= "Country_Flag"
doc_final_country.columns = cols
doc_final_country.head()


# 엑셀파일 저장
doc_final_country.to_csv("../data/covid/final_covid_data_for_graph.csv")

doc_final_country.columns

Index(['Country_Region', 'Country_Flag', '1/22/2020', '1/23/2020', '1/24/2020',
       '1/25/2020', '1/26/2020', '1/27/2020', '1/28/2020', '1/29/2020',
       ...
       '7/19/2020', '7/20/2020', '7/21/2020', '7/22/2020', '7/23/2020',
       '7/24/2020', '7/25/2020', '7/26/2020', '7/27/2020', '7/28/2020'],
      dtype='object', length=191)